Dataset rekomendasi buku resep makanan

import dataset dan library

In [117]:
import pandas as pd
import numpy as np

df = pd.read_excel('.xlsx')
print(df.head())

   user_id  recipe_id  rating           dateLastModified  \
0        1     229860       5  2012-10-31T17:54:41.867\n   
1        2     229921       5   2012-11-17T16:49:10.54\n   
2        2     229940       5  2012-11-18T16:15:56.177\n   
3        2     229940       5   2012-11-19T19:12:26.82\n   
4        2     229906       5   2012-11-22T19:06:07.08\n   

                          recipe_name  
0  Chef John's Pumpkin Cinnamon Rolls  
1  Apple Chicken Sausage Brunch Braid  
2                 Chicken Ranch Dijon  
3                 Chicken Ranch Dijon  
4     Mom's Candied Yams with Caramel  


In [ ]:
!pip install scikit-surprise
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [150]:
# for cosine similarity using linear_kernel
# content_df = df[['recipe_id', 'recipe_name']]
# content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# # Use TF-IDF vectorizer to convert content into a matrix of TF-IDF features
# tfidf_vectorizer = TfidfVectorizer()
# content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])

# content_similarity = linear_kernel(content_matrix, content_matrix)

# reader = Reader(rating_scale=(1, 5))
# data = Dataset.load_from_df(df[['user_id',
#                                   'recipe_id',
#                                   'rating']], reader)

# def get_content_based_recommendations(product_id, top_n):
#     index = content_df[content_df['recipe_id'] == product_id].index[0]
#     similarity_scores = content_similarity[index]
#     similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
#     recommendations = content_df.loc[similar_indices, 'recipe_id'].values
#     return recommendations

# for Pearson correlation coefficient from library sklearn
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

content_similarity = 1 - linear_kernel(content_matrix, content_matrix)

def get_content_based_recommendations(product_id, top_n):
    index = content_df[content_df['recipe_id'] == product_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = content_df.loc[similar_indices, 'recipe_id'].values
    return recommendations


In [144]:
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

In [149]:
def get_hybrid_recommendations(user_id, product_id, top_n):
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
    # Combine the recommendations from both approaches
    all_recommendations = np.concatenate([content_based_recommendations, collaborative_filtering_recommendations])
    # Get unique product IDs using numpy.unique
    hybrid_recommendations_ids = np.unique(all_recommendations)
    # Convert unique product IDs to product names
    hybrid_recommendations = content_df[content_df['recipe_id'].isin(hybrid_recommendations_ids)].drop_duplicates(subset=['recipe_id'])
    # Extract the desired columns
    hybrid_recommendations = hybrid_recommendations[['recipe_id', 'recipe_name']]

    return hybrid_recommendations[:top_n]

In [148]:
user_id = 1
product_id = 229875
top_n = 10
recommendations = get_hybrid_recommendations(user_id, product_id, top_n)

print(f"Hybrid Recommendations for User {user_id} based on Product {product_id}:")
# print(recommendations)
for i, row in recommendations.iterrows():
    print(f"recipe_id: {row['recipe_id']}, Product Name: {row['recipe_name']}")

Hybrid Recommendations for User 1 based on Product 229875:
recipe_id: 229921, Product Name: Apple Chicken Sausage Brunch Braid
recipe_id: 229923, Product Name: Roasted Sweet Potatoes and Vegetables With Thyme and Maple Syrup
recipe_id: 229993, Product Name: White Sauce for Pasta
recipe_id: 230132, Product Name: Chef John's Pumpkin Pie
recipe_id: 230134, Product Name: Crisp Marinated Cucumbers
recipe_id: 230137, Product Name: Cheesy Mashed Potatoes with Cubed Ham
recipe_id: 230141, Product Name: Slow Cooker Chicken and Dressing
recipe_id: 230159, Product Name: Sweet, Salty, Spicy Party Nuts
recipe_id: 230228, Product Name: Stuffed Cornish Hens Burgundy
recipe_id: 230281, Product Name: World's Best Oreo® Fudge
